In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
# Read the text file
with open('en_US.blogs.txt', 'r', encoding='utf-8') as file:
    text = file.read()


In [2]:
print(len(text))
text = text[200000:300000]
len(text)

207723793


100000

In [3]:
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)
total_words = len(tokenizer.word_index) + 1
total_words

30393

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
with open('max_sequence_len.pkl', 'rb') as f:
    max_sequence_len = pickle.load(f)
max_sequence_len

681

In [6]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
max_sequence_len

681

In [8]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [9]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [10]:
print(len(X))
print(len(y))

17843
17843


In [11]:
model = load_model("baseModel.h5")
# model = Sequential()
# model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
# model.add(LSTM(150), kernel_regularizer=l2(0.01))
# model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 680, 100)          3039300   
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 batch_normalization (BatchN  (None, 150)              600       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 30393)             4589343   
                                                                 
Total params: 7,779,843
Trainable params: 7,779,543
Non-trainable params: 300
____________________________________________

In [12]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X,y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp,Y_temp, test_size=0.5, random_state=42)
len(X_train)

14274

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=30, verbose=1, validation_data=(X_val, Y_val))

Epoch 1/30
447/447 [==============================] - 33s 66ms/step - loss: 8.9172 - accuracy: 0.0535 - val_loss: 7.4308 - val_accuracy: 0.0830
Epoch 2/30
447/447 [==============================] - 29s 64ms/step - loss: 6.7705 - accuracy: 0.0902 - val_loss: 7.6617 - val_accuracy: 0.1015
Epoch 3/30
447/447 [==============================] - 29s 65ms/step - loss: 6.1258 - accuracy: 0.1065 - val_loss: 7.8330 - val_accuracy: 0.0947
Epoch 4/30
447/447 [==============================] - 28s 63ms/step - loss: 5.4673 - accuracy: 0.1297 - val_loss: 8.2115 - val_accuracy: 0.0908
Epoch 5/30
447/447 [==============================] - 28s 64ms/step - loss: 4.5876 - accuracy: 0.1884 - val_loss: 8.9567 - val_accuracy: 0.0779
Epoch 6/30
447/447 [==============================] - 28s 63ms/step - loss: 3.5802 - accuracy: 0.3139 - val_loss: 9.7985 - val_accuracy: 0.0757
Epoch 7/30
447/447 [==============================] - 29s 65ms/step - loss: 2.7409 - accuracy: 0.4522 - val_loss: 10.4775 - val_accuracy

In [18]:
seed_text = "Hello how are you"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 46ms/step
Hello how are you in your post


In [19]:
model.save("baseModel.h5")